In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
# moved everything to HW4 path for convenience
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'HW4/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
# Load the autoreload extension
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

['EECS448_HW4_CNNs.ipynb', 'balanced_features.parquet', 'EECS448_HW4_Fine_Tuning_DistilBERT.ipynb', 'Copy of CNNs.ipynb']


In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import keras
from tensorflow.keras.layers import Conv1D, Dropout, Dense, MaxPooling1D, Flatten

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


<h3>Loading in data</h3>

In [ ]:
merged_data = pd.read_parquet('drive/My Drive/HW4/balanced_features.parquet') #pd.read_parquet("balanced_features.parquet")
#print(merged_data)
mod_data = pd.DataFrame(merged_data[['pid','x_mean', 'y_mean', 'z_mean','TAC_class']])
g_data = mod_data.groupby('pid')
print(g_data.size())

pid
BK7610     7975
BU4707     5105
CC6740     9955
DC6359     8145
DK3500     5301
HV0618     8550
JB3156    10167
JR8022     8522
MC7070    10903
MJ8002     7912
PC6771     6197
SA0297    11630
SF3079    10301
dtype: int64


In [ ]:
merged_data = pd.read_parquet('drive/My Drive/HW4/balanced_features.parquet')
merged_data = merged_data[['pid','x_mean', 'y_mean', 'z_mean','TAC_class']]

# Determine the desired number of observations per group
target_size = 5000  # Adjust this value as needed

# Group the data by 'pid'
g_data = merged_data.groupby('pid')
print(g_data.first())

# Initialize an empty list to store balanced dataframes
balanced_dfs = []

# Loop through each group
for pid, group_data in g_data:
    # Check the size of the group
    group_size = len(group_data)

    # If the group size is less than the target size, duplicate observations with replacement
    if group_size < target_size:
        balanced_group = group_data.sample(n=target_size, replace=True)
    # If the group size is greater than or equal to the target size, randomly select target_size observations
    else:
        balanced_group = group_data.sample(n=target_size, replace=False)

    # Append the balanced group to the list
    balanced_dfs.append(balanced_group)

# Combine the balanced dataframes into a single dataframe
balanced_data = pd.concat(balanced_dfs)

# Now each group should have approximately the same number of observations (target_size)
print(balanced_data.shape)
print(balanced_data.head())


          x_mean    y_mean    z_mean  TAC_class
pid                                            
BK7610  0.069174 -0.005419  0.004442        0.0
BU4707  0.004479  0.014577  0.026203        0.0
CC6740  0.007470 -0.003421  0.127106        0.0
DC6359  0.010121 -0.002451 -0.015782        0.0
DK3500  0.093050 -0.013486  0.010708        0.0
HV0618 -0.006018  0.015846  0.013143        0.0
JB3156 -0.286380  1.031547  0.954274        0.0
JR8022 -0.017745  0.013209 -0.006690        0.0
MC7070 -0.042680 -0.063925  0.004336        0.0
MJ8002 -0.009300  0.032400 -0.005600        0.0
PC6771  0.000430  0.008641 -0.003887        0.0
SA0297 -0.222442 -0.075598 -0.015889        0.0
SF3079 -0.001688  0.002147  0.024441        0.0
(65000, 5)
      pid    x_mean    y_mean    z_mean  TAC_class
0  BK7610  0.002411  0.003336  0.004677        1.0
0  BK7610 -0.091460  0.216410 -0.132584        1.0
0  BK7610 -0.003665  0.017479 -0.013882        1.0
0  BK7610  0.010805  0.007822  0.013975        0.0
0  BK7610 -0.0

In [ ]:
y = balanced_data['TAC_class']
print(y)
print(y.shape)
X = balanced_data.drop(columns=['TAC_class'])
X = X.set_index('pid')
print(X.shape)
#print(X.head())

0    1.0
0    1.0
0    1.0
0    0.0
0    0.0
    ... 
0    0.0
0    0.0
0    0.0
0    1.0
0    0.0
Name: TAC_class, Length: 65000, dtype: float64
(65000,)
(65000, 3)


In [ ]:
pid = {'BK7610': (X.loc['BK7610'], y[0:5000]),
'BU4707': (X.loc['BU4707'], y[5000:10000]),
'CC6740': (X.loc['CC6740'], y[10000:15000]),
'DC6359': (X.loc['DC6359'], y[15000:20000]),
'DK3500': (X.loc['DK3500'], y[20000:25000]),
'HV0618': (X.loc['HV0618'], y[25000:30000]),
'JB3156': (X.loc['JB3156'], y[30000:35000]),
'JR8022': (X.loc['JR8022'], y[35000:40000]),
'MC7070': (X.loc['MC7070'], y[40000:45000]),
'MJ8002': (X.loc['MJ8002'], y[45000:50000]),
'PC6771': (X.loc['PC6771'], y[50000:55000]),
'SA0297': (X.loc['SA0297'], y[55000:60000]),
'SF3079': (X.loc['SF3079'], y[60000:65000])}

In [ ]:
print(pid['SF3079'][0])
print(pid['SF3079'][1])

          x_mean    y_mean    z_mean
pid                                 
SF3079  0.007301 -0.009005  0.037025
SF3079  0.013848 -0.003048  0.009462
SF3079 -0.000200  0.012300  0.055400
SF3079 -0.002482 -0.003617 -0.013531
SF3079  0.176452  0.098242  0.043821
...          ...       ...       ...
SF3079 -0.000200  0.012300  0.055400
SF3079 -0.000200  0.012300  0.055400
SF3079 -0.000200  0.012300  0.055400
SF3079 -0.022796 -0.011320  0.025474
SF3079 -0.000200  0.012300  0.055400

[5000 rows x 3 columns]
0    1.0
0    1.0
0    0.0
0    1.0
0    0.0
    ... 
0    0.0
0    0.0
0    0.0
0    1.0
0    0.0
Name: TAC_class, Length: 5000, dtype: float64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=15000, shuffle=False)

In [ ]:
X_idx = X_train.set_index('pid')
y_idx = y_train[0:5000]

X2_idx = X_test.set_index('pid')
y2_idx = y_test[0:5000]
# BK7610 pid
BK7610_X_train = X_idx.loc['BK7610']
print(BK7610_X_train)
#print(y_idx)

# PC6771 pid
PC6771_X_test = X2_idx.loc['PC6771']
print(PC6771_X_test)

          x_mean    y_mean    z_mean
pid                                 
BK7610  0.004804  0.000036  0.002485
BK7610  0.000872  0.000761  0.008514
BK7610  0.003516 -0.011094  0.013558
BK7610  0.010000  0.029500 -0.006800
BK7610  0.022207 -0.014972  0.009713
...          ...       ...       ...
BK7610  0.001109 -0.002541 -0.004961
BK7610 -0.003353  0.001035  0.003076
BK7610 -0.001449 -0.001587  0.019304
BK7610  0.005500  0.003000  0.031300
BK7610 -0.003811  0.002456  0.006373

[5000 rows x 3 columns]
          x_mean    y_mean    z_mean
pid                                 
PC6771  0.003116  0.000843 -0.014316
PC6771 -0.014000 -0.026400 -0.046900
PC6771 -0.014000 -0.026400 -0.046900
PC6771 -0.039400 -0.005300 -0.001300
PC6771  0.017067  0.000887  0.006969
...          ...       ...       ...
PC6771  0.017658 -0.012468 -0.010895
PC6771 -0.014000 -0.026400 -0.046900
PC6771  0.001300 -0.009400 -0.045200
PC6771  0.009985  0.014157  0.001938
PC6771 -0.014000 -0.026400 -0.046900

[5000 rows x

In [ ]:
#SAMPLE!
#drop pid
#BK7610_X_train = BK7610_X_train.drop(columns=['pid'])
#PC6771_X_test = PC6771_X_test.drop(columns=['pid'])
for k in pid:
  print(pid[k][0])
  pid[k][0] = np.expand_dims(pid[k][0], axis=0)
  pid[k][1] = np.expand_dims(pid[k][1], axis=0)

  #print(X_train)
  #print(X_train_reshaped.shape)
  #print(X_test)
  #print(X_test_reshaped.shape)

  #y_train = np.expand_dims(y_idx, axis=0)
print(pid['BK7610'][1].shape)
  #y_test = np.expand_dims(y2_idx, axis=0)
  #print(y_test.shape)

          x_mean    y_mean    z_mean
pid                                 
BK7610 -0.012911  0.004477  0.002180
BK7610 -0.006431 -0.008271 -0.005228
BK7610  0.016802  0.003937 -0.030708
BK7610  0.001200  0.004711  0.006755
BK7610 -0.007077  0.004037  0.008237
...          ...       ...       ...
BK7610 -0.009787  0.022891  0.055548
BK7610 -0.000959 -0.001126 -0.010042
BK7610 -0.329786 -0.061911 -0.131287
BK7610  0.005500  0.003000  0.031300
BK7610  0.010000  0.029500 -0.006800

[5000 rows x 3 columns]


TypeError: 'tuple' object does not support item assignment

In [ ]:
#drop pid
X_train = X_train.drop(columns=['pid'])
X_test = X_test.drop(columns=['pid'])

X_train_reshaped = np.expand_dims(X_train, axis=0)
X_test_reshaped = np.expand_dims(X_test, axis=0)

#print(X_train)
print(X_train_reshaped.shape)
#print(X_test)
print(X_test_reshaped.shape)

y_train = np.expand_dims(y_train, axis=0)
print(y_train.shape[0])
y_test = np.expand_dims(y_test, axis=0)
print(y_test.shape)

1
(1, 5000)
(1, 5000, 3)
(1, 5000, 3)


In [ ]:
print(X_train[0:5000])

      x_mean    y_mean    z_mean
0   0.004804  0.000036  0.002485
0   0.000872  0.000761  0.008514
0   0.003516 -0.011094  0.013558
0   0.010000  0.029500 -0.006800
0   0.022207 -0.014972  0.009713
..       ...       ...       ...
0   0.001109 -0.002541 -0.004961
0  -0.003353  0.001035  0.003076
0  -0.001449 -0.001587  0.019304
0   0.005500  0.003000  0.031300
0  -0.003811  0.002456  0.006373

[5000 rows x 3 columns]


<h3>CNN Model</h3>

In [ ]:
# fit and evaluate a model
n_timesteps, n_features, n_outputs = 5000, 3, 5000
model = keras.models.Sequential()
model.add(Conv1D(filters=3, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Conv1D(filters=3, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
#model.add(Dense(10, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# fit network
#model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluate model
#print(X_test.shape)
#print(y_test.shape)
#_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
#return accuracy

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4998, 3)           30        
                                                                 
 conv1d_1 (Conv1D)           (None, 4996, 3)           30        
                                                                 
 dropout (Dropout)           (None, 4996, 3)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 2498, 3)           0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 7494)              0         
                                                                 
 dense (Dense)               (None, 5000)              37475000  
                                                        

In [ ]:
#evaluate_model(X_train_reshaped, y_train, X_test_reshaped, y_test)
verbose, epochs, batch_size = 0, 10, 5000
#for k in pid:
  #if k != 'MC7070':
X_train = np.expand_dims(pid['BK7610'][0], axis=0)
y_train = np.expand_dims(pid['BK7610'][1], axis=0)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)


X_test = np.expand_dims(pid['MC7070'][0], axis=0)
y_test = np.expand_dims(pid['MC7070'][1], axis=0)
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
print(f"Accuracy is:{accuracy}")

Accuracy is:0.0


In [ ]:
print(X_train)

[[[ 0.002411   0.0033364  0.0046772]
  [-0.0914597  0.2164096 -0.1325844]
  [-0.0036652  0.0174794 -0.0138823]
  ...
  [ 0.1226    -0.2164    -0.0361   ]
  [ 0.000603  -0.0038627  0.0043433]
  [ 0.002582   0.0019319  0.0054821]]]


In [ ]:
print(y_train)

[[1. 1. 1. ... 0. 0. 1.]]


In [ ]:
print(history.history['accuracy'])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
print(history.history['loss'])

[0.537026047706604, 0.5112406015396118, 0.5209829807281494, 0.4994974136352539, 0.48767173290252686, 0.484801709651947, 0.48040008544921875, 0.46471595764160156, 0.4856460392475128, 0.4864591658115387]
